In [ ]:
import numpy as np
from itertools import permutations
import matplotlib.pyplot as plt
import matplotlib.pyplot as mpl
import scipy.io
import sys
sys.path.insert(0,'..')
from analysis.analysis_utils import *
%matplotlib inline

## Load data

In [ ]:
my_data = scipy.io.loadmat('../data/dataphaseorder_selunitsvex.mat')
my_data = my_data['matrix']
freqs = [1.5, 1.75, 2.04, 2.37, 2.75, 3.21, 3.73, 4.35, 5.06, 5.90, 6.87, 8]
pltcolors, pltcolors_alt = steffiscolours()


In [ ]:
plt.scatter(np.arange(len(my_data)),my_data[:,0], 
            color = pltcolors_alt[-1])
plt.ylabel("Freqs")
plt.xlabel("Neuron index");
print("Freqs from " + str(np.min(my_data[:,0])) +
      " to " + str(np.max(my_data[:,0])) + " Hz")

## Predictions

In [ ]:
#Create ground truth
#---------------------

#0: (3, 1, 2)
#1: (1, 3, 2)
#2: (3, 2, 1)
#3: (2, 3, 1)
#4: (1, 2, 3)
#5: (2, 1, 3)

perms =[[3,1,2],[1,3,2],[3,2,1],[2,3,1],[1,2,3],[2,1,3]]


#First frequency slots:
#(1.25, 1.67) : 5: (2, 1, 3)
#(1.67, 2.50) : 2: (3, 2, 1)
#(2.50, 3.33) : 4: (1, 2, 3)
#(3.33, 3.75) : 0: (3, 1, 2)

orders = np.array([5,2,4,0])

fmin =1
fmax = 9

#Transition between orders at solutions of:
#e^(i2pi isi f) = 0
#e^(i4pi isi f) = 0
#e^(i6pi isi f) = 0

boundaries = []
for i in range(1,int(np.ceil(fmax/0.4))):
    boundaries.append(i*1/0.4)
    boundaries.append(i*1/0.8)
    boundaries.append(i*1/1.2)
    
#find unique solutions within range and sort them    
boundaries = np.sort(np.unique(boundaries))
boundaries = boundaries[boundaries<fmax]
boundaries = boundaries[boundaries>fmin]
print(boundaries)

#make prediction matrix
prediction = np.zeros((len(boundaries)-1,6))
for i in range(len(boundaries)-1):
    prediction[i,orders[i%4]]=1

In [ ]:
labels = ["{:.2f}".format(boundaries[i]).strip("0")for i in np.arange(len(boundaries))[::2]]

plt.imshow(prediction.T,origin='lower')
plt.xticks(np.arange(len(boundaries))[::2]-0.5, labels = labels);
plt.yticks(np.arange(6), labels = perms);


## Phase order in data

In [ ]:
# Calculate circ shifted order of phases in data
data = np.zeros((len(boundaries)-1,6))
num_neurons = len(my_data)

for neuron in range(num_neurons):
    for i in range(len(boundaries)-1):
        if my_data[neuron,0]>boundaries[i] and my_data[neuron,0]<boundaries[i+1]:
            avgs = my_data[neuron,1:]
            #Set position 0 to be first in phase
            avgs -= avgs[0]
            #Circularly shift the neurons that now have a negative position
            avgs[avgs < 0] += 4
            for permi, perm in enumerate(perms):
                if (avgs[1:] == np.array(perm)).all():
                    data[i,permi]+=1

plt.imshow(data.T,origin='lower')
plt.xticks(np.arange(len(boundaries-1))[::2]-0.5, labels = labels);
pred = np.sum((data*prediction))/np.sum(data)

## Compare predictions of data to permutation

In [ ]:
scores= []
pred_perc = 0
num_trials = 10000
fk_data = np.copy(my_data)
ind =np.arange(len(my_data))
num_neurons = len(my_data)

for i in range(num_trials):
    if i%1000==0:
        print(str(i/num_trials*100) + "%")
    np.random.shuffle(ind)
    fk_data[:,0]=fk_data[ind,0]
    data = np.zeros((len(boundaries)-1,6))
    for neuron in range(num_neurons):
        for i in range(len(boundaries)-1):
            if fk_data[neuron,0]>boundaries[i] and fk_data[neuron,0]<boundaries[i+1]:
                avgs = fk_data[neuron,1:]
                #Set position 0 to be first in phase
                avgs -= avgs[0]
                #Circularly shift the neurons that now have a negative phase
                avgs[avgs < 0] += 4
                for permi, perm in enumerate(perms):
                    if (avgs[1:] == np.array(perm)).all():
                        data[i,permi]+=1

    score=np.sum((data*prediction))/np.sum(data)
    scores.append(score)
    if score>pred:
        pred_perc += 1
pred_perc/=num_trials
print(pred_perc)
scores = np.array(scores)

In [ ]:
with mpl.rc_context(fname="matplotlibrc"):
    plt.figure(figsize = (1.5,1.5))
    plt.hist(np.array(scores), bins = 24, color = pltcolors_alt[1], label ='permutations')
    plt.axvline(pred, color ='black', label ='data')
    plt.ylabel("counts")
    plt.xlabel("prop. predicted correctly")
    plt.xticks(np.arange(0,4/12,1/12), 
               labels = ["{:.2f}".format(np.arange(0,4/12,1/12)[i]) for i in range(len(np.arange(0,4/12,1/12)))])
    plt.legend(bbox_to_anchor = (0.9,1.1))